In [2]:
from funcs_mrcio import iwrhdr_opened, irdhdr_opened, iwrsec_opened, irdpas_opened
from funcs_mrcio import irdsec_opened
import matplotlib.pyplot as plt
import numpy as np

from skimage.exposure import rescale_intensity

In [9]:
#use loop to iterate through the .txt file that contain the names of all the micrographs
#with open("yourfilename.txt", 'r') as mlist:

micrographnamein= "test_lattice.mrc"
#print(micrographnamein[0:-1])
micrographnameout= filename.replace(".mrc", "_filt.mrc")
psize = 1.45 # Angstroms/pixel

#print(micrographnamein)

micrograph = open(micrographnamein,'rb')
image = rescale_intensity(irdsec_opened(micrograph,0)) #rescale to range (-1,1)
header = irdhdr_opened(micrograph)
nx = header['nx']; ny = header['ny']; nz = header['nz']
#print(nx,ny,nz)

psize = 1.45 # Angstroms/pixel
#plt.rcParams['figure.figsize'] = [20, 10]
#plt.imshow(image,cmap='gray')
#plt.show()
boxsize = np.maximum(nx,ny)
image_padded=np.empty([boxsize,boxsize])
image_padded[:,:]=np.mean(image)
image_padded[0:nx,0:ny]=image
#plt.rcParams['figure.figsize'] = [20, 40]
#plt.imshow(image_padded,cmap='gray')
#plt.show()

# Set up the FFTW Fourier Transform
import pyfftw   
fftmod = pyfftw.interfaces.numpy_fft
pyfftw.interfaces.cache.enable()
#enabling the cache that temporarily stores a copy of any interim py.fftw.fftw objects
import multiprocessing
#        fft_threads = multiprocessing.cpu_count()

imagefftw  = pyfftw.empty_aligned([boxsize,boxsize],dtype=np.float64)
#specifies that the input is real. This will save processing power because FFT's of real numbers will be Hermitian
print("Using FFTW with this many threads")
#        print(fft_threads)

imagefftw[:,:]=image_padded
#        imagec=fftmod.rfftn(imagefftw,threads=fft_threads,planner_effort="FFTW_ESTIMATE")
imagec=fftmod.rfftn(imagefftw,threads=1,planner_effort="FFTW_ESTIMATE")

#finding the shape of the fourier transformed image
ny_fft, nx_fft= np.shape(imagec)

from skimage import filters

power        = np.log(np.real(imagec * np.conj(imagec)))
power_smooth = filters.gaussian(power,9)
peaky        = power-power_smooth
peaky_smooth = filters.gaussian(peaky,1)

#finding the threshold by setting 3*standard deviation + the mean
threshold    = 3*np.std(peaky_smooth)+filters.threshold_mean(peaky_smooth) 
print(threshold)
#finding the index where the power spectrum has a higher value than the threshold. Exclude the right most part of
#the fourier transform because of a band of high values at the highest frequencies
masked_index_y, masked_index_x= np.where(peaky_smooth[:, 0:7*np.int(nx_fft/8)]>threshold)

masked= power.copy()

#dividing the Fourier transform image into two quadrants and getting the index for each quadrant
first_half= np.where(masked_index_y<(ny_fft/2))[0]
second_half= np.where(masked_index_y>(ny_fft/2))[0]
masked_index_y_first= masked_index_y[first_half]
masked_index_y_second= masked_index_y[second_half]


#set the shift into 1% of the size of the Fourier image
X= np.int(0.01*nx_fft)
Y= np.int(0.01*ny_fft)

#shift the index, right and down for the first quadrant, and right and up for the second quadrant
masked_index_y_shifted= np.append(masked_index_y_first+Y, masked_index_y_second-Y)
masked_index_x_shifted= masked_index_x+X

#change the values that are higher than the threshold into the shifted values
masked[masked_index_y, masked_index_x]= power[masked_index_y_shifted, masked_index_x_shifted]

#show = power
#print(show.shape)
#print(np.int(boxsize/4))
#bottomoffft = show[0:np.int(boxsize/8),0:np.int(boxsize/8)]
#bottomoffft = show[0:np.int(boxsize/8), 0: np.int(boxsize/8)]
#plt.rcParams['figure.figsize'] = [20, 10]
#plt.imshow(bottomoffft,cmap='gray', vmin= 0, vmax= 25)
#plt.show()    

#show = masked
#print(show.shape)
#print(np.int(boxsize/4))
#bottomoffft = show[0:np.int(boxsize/8),0:np.int(boxsize/8)]
#bottomoffft = show[0:np.int(boxsize/8),0:np.int(boxsize/8)]
#plt.rcParams['figure.figsize'] = [20, 10]
#plt.imshow(bottomoffft,cmap='gray', vmin= 0, vmax= 25)
#plt.show()

#creating the random phase
phase_array= np.random.uniform(low= 0.0, high= 2*np.pi, size= np.shape(imagec))
j= 0.0+1j

#copying fourier transformed image
imagecmasked = imagec.copy()
#adding a random phase to the fourier transformed image
imagecmasked_random_phase= imagec.copy()*np.exp(phase_array*j)

#masking out the diffraction spots by replacing the peaks with the values (with added random phase) from the shifted axis
imagecmasked[masked_index_y, masked_index_x]= imagecmasked_random_phase[masked_index_y_shifted,masked_index_x_shifted]

# calculate inverse FFT of masked FTi
image_nolattice_padded=fftmod.irfftn(imagecmasked,threads=fft_threads,planner_effort='FFTW_ESTIMATE')

#extract micrograph from the padded, masked
image_nolattice=image_nolattice_padded[0:nx,0:ny]

#plt.rcParams['figure.figsize'] = [20, 40]
#print(np.min(image_nolattice),np.max(image_nolattice))
#plt.imshow(image_nolattice,cmap='gray')
#plt.show()

filteredmic = open(micrographnameout[0:-1],'wb')
nxyz  = np.array([nx,ny,1],dtype=np.float32)
dmin  = np.min(image_nolattice)
dmax  = np.max(image_nolattice)
dmean = np.sum(image_nolattice)/(nx*ny)
iwrhdr_opened(filteredmic,nxyz,dmin,dmax,dmean,mode=2)
iwrsec_opened(image_nolattice,filteredmic)

Using FFTW with this many threads
1.1515480052329694


/tmp/ipykernel_10060/381287831.py:61: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  masked_index_y, masked_index_x= np.where(peaky_smooth[:, 0:7*np.int(nx_fft/8)]>threshold)
/tmp/ipykernel_10060/381287831.py:73: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note lin

IndexError: shape mismatch: indexing arrays could not be broadcast together with shapes (5772,) (5774,) 